In [1]:
from ultralytics import YOLO
import cv2
import numpy as np

In [2]:
model=YOLO('./YOLO weights/yolov8n.pt')

In [51]:
### creating mask for image 

blank=np.zeros((480,640),dtype=np.uint8)
mask=cv2.rectangle(blank,(0,180),(640,320),(255,255,255),cv2.FILLED)


In [56]:
import math

cap=cv2.VideoCapture('.\images\car_moving.mp4')

counter=0

while True:
    ret,img=cap.read()
    if ret:
        img=cv2.resize(img,(640,480))
        img_crop=cv2.bitwise_and(img,img,mask=mask) ##aplying mask 
        results=model(img_crop, stream=True,verbose=False)

        for r in results:
            boxes=r.boxes
            for box in boxes:
                x1,y1,x2,y2=box.xyxy[0] #xywh can also be used
                x1,y1,x2,y2=int(x1),int(y1),int(x2),int(y2)


                conf=math.ceil(box.conf[0]*100)/100
                classs=int(box.cls[0])
                if classs==2 and conf>0.3:
                    ##left side cars
                    if x1<=260:
                        if (y2>=248 and y2<=250):
                            counter+=1
                        else:
                            continue
                    ##right side cars 
                    else:
                        if(y1>=250 and y1<=252):
                            counter+=1
                        else:
                            continue
                    cv2.rectangle(img,(x1,y1),(x2,y2),(255,0,255),3)
                    
        cv2.rectangle(img,(0,20),(200,60),(255,100,100),cv2.FILLED)
        cv2.putText(img,f'counter= {counter}',(0,50),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,200),3)

        cv2.line(img,(0,250),(640,250),(0,220,0),2) ## line to be crossed
        cv2.line(img,(260,180),(260,320),(0,220,0),2) ## dividing left and right
        cv2.imshow("Img",img_crop)
        cv2.imshow("Original",img)
        if cv2.waitKey(1) & 0xFF==ord('q'):
            break
    else:
        break
cap.release()
cv2.destroyAllWindows()

print('No. of cars were ',counter)

No. of cars were  19
